In [12]:
import os
import json
import urllib.request

import sys
sys.path.append("/home/etaylor/pycharmProjects/thesis/notebooks/fast-labeling-workflow/detectron2")
sys.path.append("/home/etaylor/pycharmProjects/thesis/notebooks/fast-labeling-workflow")

import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode

import numpy as np
import cv2
import matplotlib.pyplot as plt

from pycocotools import mask

from segments.utils import export_dataset
import utils

In [20]:
cfg = get_cfg()

#Load model config and pretrained model
instance_segmentation_model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
detection_model = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(instance_segmentation_model))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(instance_segmentation_model)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = 'cuda'

predictor = DefaultPredictor(cfg)

In [21]:
class Model:
    def __init__(self, predictor):
        self.predictor = predictor

    def _convert_to_segments_format(self, image, outputs):
        # https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
        segmentation_bitmap = np.zeros((image.shape[0], image.shape[1]), np.uint32)
        annotations = []
        counter = 1
        instances = outputs['instances']
        for i in range(len(instances.pred_classes)):
            category_id = int(instances.pred_classes[i])
            instance_id = counter
            mask = instances.pred_masks[i].cpu()
            segmentation_bitmap[mask] = instance_id
            annotations.append({'id': instance_id, 'category_id': category_id})
            counter += 1
        return segmentation_bitmap, annotations

    def __call__(self, image):
        image = np.array(image)
        outputs = self.predictor(image)
        label, label_data = self._convert_to_segments_format(image, outputs)

        return label, label_data

In [22]:
image = cv2.imread('/home/etaylor/images/trichomes_patches/week5_3xzoom_regular_best_images/IMG_0003_p26.jpg')
model = Model(predictor)
pred, bitmap = model(image)

In [25]:
bitmap

[]